Answer 1

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
#import libraries
import re
import pandas as pd

In [2]:
#import dataset
#df = pd.read_csv("/content/drive/My Drive/coupons_ner.csv",header = None)
df = pd.read_csv("coupons_ner.csv",header = None)
df.columns = ['coupons']

In [3]:
#function to identify all the required values
def value(df):
    values = []
    for i in df.coupons.values:
        # Noticed that all of the value are in either one of the 4 formats
        # $ 5.00, $5.00, $ 5, $.55
        #* - 0 or more #+ - 1 or more  #? - 0 or 1
        #pattern = re.findall(r'\$\s?\.?\d+\.?[\d+ ]',i)         
        pattern = re.findall(r'\$\s?\.?\d+\.?\d*\s?',i)
        #check for dollar, check for space, check for dot,check for digits, check for dot, check for digits    
        pattern2 = re.findall(r'\d+[¢]',i)     
        #test use case for cents                   
        try:
            values.append(pattern.pop(0))
        except IndexError:
            try:
                values.append(pattern2.pop(0))
            except IndexError:
                values.append('0')
        
    return values

In [4]:
x = value(df)

In [5]:
df['facevalue'] = x

In [6]:
#final dataframe
df

,coupons,facevalue
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...,$2.00
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...,$2.00
2,Save $2.00 ONE Tide Laundry Detergent (exclude...,$2.00
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...,$1.00
4,$3.00 OFF when you purchase any THREE (3) Pepp...,$3.00
...,...,...
881,SAVE $2.00 when you buy $7 worth of participat...,$2.00
882,SAVE $5.00 on any Gain scented purchase of $30...,$5.00
883,SAVE $1.00 when you buy any ONE (1) NEW AXE pr...,$1.00
884,BUY TWO GET ONE Buy ONE (1) CHIPS AHOY! Cookie...,0


In [0]:
#indices = [i for i, j in enumerate(x) if j == '0']
#print(df['coupons'].values[157])

In [0]:
#for i in indices:
    #print(df['coupons'].values[i])

$1 Off Your Purchase of $1 or More *Standard exclusions apply.


**Answer 2**

In [2]:
#download spacy english statistical module
#! python -m spacy download en_core_web_md

In [1]:
#pip install -U spacy==2.0.0

In [0]:
import spacy
#nlp = spacy.load('en_core_web_md')

In [0]:
string = df['coupons'].values[0]

In [5]:
string

'Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size).'

In [0]:
doc = nlp(string)

In [23]:
doc

Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size).

In [24]:
# check using the  en_core_Web_md statistical english package to see if it can identify product entities
for ent in doc.ents:
    print('Entity: %s, Label: %s'%(ent.text, ent.label_))

Entity: 72, Label: CARDINAL
Entity: Downy, Label: PRODUCT
Entity: 48 oz, Label: QUANTITY
Entity: Downy Wrinkle Guard, Label: ORG
Entity: Downy Nature Blends, Label: PRODUCT
Entity: 67 oz, Label: QUANTITY
Entity: Bounce/Downy Wrinkle Guard, Label: ORG
Entity: 8.6 oz, Label: QUANTITY
Entity: Downy Unstopables, Label: PERSON
Entity: Fresh Protect, Odor, Label: PERSON
Entity: Downy Libre Enjuague, Label: ORG
Entity: Gain Fireworks, Label: ORG


In [25]:
product = []
for ent in doc.ents:
    if ((ent.label_ == 'PRODUCT') or (ent.label_ == 'ORG')) and len(ent.text) > 5:
      print(ent.text)
      product.append(ent.text)

Downy Wrinkle Guard
Downy Nature Blends
Bounce/Downy Wrinkle Guard
Downy Libre Enjuague
Gain Fireworks


When we try to identify entities using the en_core_web_md module, we see that it did not do a very good job. Hence, we will try to create our own dataset and try tot train the model on it.

**Training the model**

In [0]:
import random
import spacy

In [6]:
df['coupons'].values[2]

'Save $2.00 ONE Tide Laundry Detergent (excludes Tide Purclean Laundry Detergent, Studio by Tide Laundry Detergent, Tide PODS, Tide Rescue, Tide Simply Laundry Detergent, Tide Simply PODS, Tide Detergent 10 oz and trial/travel size).'

In [76]:
#fucntion to check the indices of the product entities in coupon data. This will be used in the cell after this.
import re
def substring_range(s, substring):
    for i in re.finditer(substring, s):
        yield (i.start(), i.end())

s = "Save $1.00 on BENADRYL® product when you buy ONE (1) BENADRYL®  20ct, 24ct or 4oz product. Excludes topical, trial &amp; travel sizes"
substring = "BENADRYL"
print([x for x in substring_range(s, substring)])

[(14, 22), (53, 61)]


In [0]:
#get the train data ready
TRAIN_DATA = [
         ('Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size).',{
             'entities':[(15,46,'PRODUCT'),(73,91,'PRODUCT'),(111,130,'PRODUCT'),(244,263,'PRODUCT'),(150,169,'PRODUCT'),(284,306,'PRODUCT'),(334,351,'PRODUCT'),(407,427,'PRODUCT'),(429,443,'PRODUCT')]
             }), 
              ('Save $2.00 ONE Tide PODS OR Tide Power PODS (excludes Tide Liquid/Powder Laundry Detergent, Tide Simply, Tide Simply PODS, and Tide PODS 9 ct and below, and trial/travel size).',{
             'entities':[(15,24,'PRODUCT'),(127,136,'PRODUCT'),(28,43,'PRODUCT'),(54,90,'PRODUCT'),(91,103,'PRODUCT'),(104,116,'PRODUCT')]
             }),
              ('Save $2.00 ONE Tide Laundry Detergent (excludes Tide Purclean Laundry Detergent, Studio by Tide Laundry Detergent, Tide PODS, Tide Rescue, Tide Simply Laundry Detergent, Tide Simply PODS, Tide Detergent 10 oz and trial/travel size).',{
             'entities':[(15,37,'PRODUCT'),(91,113,'PRODUCT'),(48,79,'PRODUCT'),(81,87,'PRODUCT'),(115,124,'PRODUCT'),(126,137,'PRODUCT'),(139,168,'PRODUCT'),(169,202,'PRODUCT')]
             }),
              ('SAVE 50¢ ON TWO when you buy TWO BOXES (8.9 - 15.2 OZ. EXCLUDES LARGE SIZE) any flavor/variety General Mills cereal listed: *',{
             'entities':[(95,115,'PRODUCT')]
             }),
              ('$3.00 OFF when you purchase any THREE (3) Pepperidge Farm® Farmhouse buns, bread, or cookies',{
             'entities':[(42,73,'PRODUCT'),(75,80,'PRODUCT'),(85,92,'PRODUCT')]
             }),
              ('BUY TWO GET ONE Buy ONE (1) CHIPS AHOY! Cookies (9.5-19.5 oz.) and ONE (1) SOUR PATCH KIDS Candy (4 oz. or larger), get ONE (1) SOUR PATCH KIDS CHIPS AHOY! Cookies FREE (8 oz.;)',{
             'entities':[(28,38,'PRODUCT'),(144,154,'PRODUCT'),(75,96,'PRODUCT')]
             }),
              ('SAVE $1.00 when you buy any ONE (1) NEW AXE product',{
             'entities':[(40,43,'PRODUCT')]
             }),
              ('SAVE $1.00 when you buy any ONE (1) NEW AXE product',{
             'entities':[(40,43,'PRODUCT')]
             }),
              ('SAVE $5.00 on any Gain scented purchase of $30 or more',{
             'entities':[(18,30,'PRODUCT')]
             }),
              ('SAVE $5.00 on any Gain scented purchase of $30 or more',{
             'entities':[(18,30,'PRODUCT')]
             }),
              ('SAVE $2.00 when you buy $7 worth of participating REESE’S brand candy',{
             'entities':[(50,55,'PRODUCT')]
             }),
              ('Save $1.00 on BENADRYL® product when you buy ONE (1) BENADRYL®  20ct, 24ct or 4oz product. Excludes topical, trial &amp; travel sizes',{
             'entities':[(14,22,'PRODUCT'),(53,61,'PRODUCT')]
             })
              
                  
]

In [32]:
! python -m spacy info


    Info about spaCy

    spaCy version      2.0.0          
    Location           /usr/local/lib/python3.6/dist-packages/spacy
    Platform           Linux-4.19.104+-x86_64-with-Ubuntu-18.04-bionic
    Python version     3.6.9          
    Models                            



In [64]:



#def main(model = None,output_dir = None,n_iter = 100):
  #load the model, set up the pipeline and train the entity recognizer
model = None
n_iter = 50  
output_dir = '/content/'
if model is not None:
  nlp = spacy.load(model)
  print("Loaded model '%s'" % model)
else:
  nlp = spacy.blank('en') # create a blank class for a particular language model
  print("Created blank 'en' model")






Created blank 'en' model


In [0]:
if 'ner' not in nlp.pipe_names:  # check for pipe names eg ['tagger','parser']
  ner = nlp.create_pipe('ner') # make sure entity recognizer is present in the pipeline
  nlp.add_pipe(ner,last = True)
else:
  ner = nlp.get_pipe('ner') #only get the entity recognition segment of the NLP pipeline

for _,annotations in TRAIN_DATA:     
  for ent in annotations.get('entities'):
    ner.add_label(ent[2]) ## add the labels - product

In [71]:
%%time
import random
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes): ##only train NER pipeline
  optimizer = nlp.begin_training()
  for itn in range(n_iter):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for text,annotations in TRAIN_DATA:
      nlp.update([text], [annotations],sgd=optimizer, drop=0.5, losses=losses)
    print(losses)



{'ner': 6.038022633529091}
{'ner': 4.542508603279626}
{'ner': 7.994899052494223}
{'ner': 5.604456961880197}
{'ner': 2.7019766436585098}
{'ner': 2.7964433931177712}
{'ner': 6.00000526070635}
{'ner': 1.8526999105758353}
{'ner': 6.0003490584184265}
{'ner': 7.998437097326331}
{'ner': 4.7258746114821975}
{'ner': 2.0073245423593473}
{'ner': 5.9967625075688265}
{'ner': 7.91037990887518}
{'ner': 1.414316728882677}
{'ner': 13.831878676243264}
{'ner': 2.000000036189035}
{'ner': 8.189670636792869}
{'ner': 4.044508703887373}
{'ner': 7.5606496117182385}
{'ner': 5.040306343469984}
{'ner': 6.000088453719707}
{'ner': 4.873600929461491}
{'ner': 9.998161800512392}
{'ner': 16.783867390081813}
{'ner': 2.116467900864006}
{'ner': 17.33012920724049}
{'ner': 7.7086274712119}
{'ner': 4.020007873996486}
{'ner': 3.971949043046322}
{'ner': 8.063876080307079}
{'ner': 2.5750090479290852}
{'ner': 3.967899450217405}
{'ner': 3.9999870761505845}
{'ner': 5.040070881125542}
{'ner': 1.7350183515786512e-05}
{'ner': 0.02362

In [72]:
%%time
products = []
for text in df['coupons'].values:###TRAIN_DATA:
  doc = nlp(text)
  #print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
  #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
  sub_pro = []
  for i in doc.ents:
    sub_pro.append(i.text)

  products.append(sub_pro)



CPU times: user 7.59 s, sys: 3.86 s, total: 11.5 s
Wall time: 5.9 s


In [73]:
products

[['Downy Liquid Fabric Conditioner',
  'Downy Odor Protect',
  'Downy Wrinkle Guard',
  'Downy Nature Blends',
  'Downy Wrinkle Guard',
  'In Wash Scent Boosters',
  'Downy Unstopables',
  'Downy Libre Enjuague',
  'Gain Fireworks'],
 ['Tide PODS',
  'Tide Power PODS',
  'Tide Liquid/Powder Laundry Detergent',
  'Tide Simply',
  'Tide Simply PODS',
  'Tide PODS'],
 ['Tide Laundry Detergent',
  'Tide Purclean Laundry Detergent',
  'Studio',
  'Tide Laundry Detergent',
  'Tide PODS',
  'Tide Rescue',
  'Tide Simply Laundry Detergent',
  'Tide Simply PODS',
  'Tide Detergent'],
 ['General Mills cereal'],
 ['Pepperidge Farm® Farmhouse buns', 'bread', 'cookies'],
 ['CHIPS AHOY'],
 ['LARGER', 'Valley™ Granola Bars', 'Biscuits'],
 ['Sargento® Natural Cheese Slices'],
 ['Oikos Greek Yogurt Cup', 'Triple Zero'],
 ['® Coffee', 'Any', 'Barista Blends'],
 ['Tide Simply Laundry Detergent',
  'Tide Simply PODS',
  'Downy Liquid Fabric Conditioner',
  'In',
  'Wash Scent Boosters',
  'Downy Unstopabl

In [0]:
df['products'] = products

In [77]:
#here we have the final result.
df.head(10)

,coupons,products
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...,"[Downy Liquid Fabric Conditioner, Downy Odor P..."
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...,"[Tide PODS, Tide Power PODS, Tide Liquid/Powde..."
2,Save $2.00 ONE Tide Laundry Detergent (exclude...,"[Tide Laundry Detergent, Tide Purclean Laundry..."
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...,[General Mills cereal]
4,$3.00 OFF when you purchase any THREE (3) Pepp...,"[Pepperidge Farm® Farmhouse buns, bread, cookies]"
5,SAVE $1.11 when you buy any ONE (1) Familly Si...,[CHIPS AHOY]
6,SAVE $1.00 ON TWO when you buy TWO PACKAGES an...,"[LARGER, Valley™ Granola Bars, Biscuits]"
7,Save $1.00 on any TWO (2) Sargento® Natural Ch...,[Sargento® Natural Cheese Slices]
8,$0.65 OFF On Any ONE (1) Oikos Greek Yogurt Cu...,"[Oikos Greek Yogurt Cup, Triple Zero]"
9,$2.00 OFF ONE (1) SMALL bag of Eight O'Clock® ...,"[® Coffee, Any, Barista Blends]"
